In [ ]:
import matplotlib.pyplot as plt
from gtsrb_loader.download_gtsrb import download_gtsrb

from skimage import data
from skimage.transform import resize
from PIL import Image

import numpy as np
import pandas as pd

import sys
import os
import fnmatch

In [ ]:
absolute_path = download_gtsrb()

In [ ]:
ppm_filelist = []
csv_filelist = []

for root, dirs, files in os.walk(absolute_path):
    ppm_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.ppm')]
    csv_filelist += [os.path.join(root, file) for file in files if fnmatch.fnmatch(file, '*.csv')]

In [ ]:
print(ppm_filelist[0])
print(os.path.split(ppm_filelist[0])[1])
print (os.path.dirname(ppm_filelist[0]))
directory =os.path.dirname(ppm_filelist[0])+'M'
print (directory)

In [ ]:
def mirrorVer(pic) :
    v=pic.shape[1]
    i=0
    picnew=np.zeros((pic.shape[0],pic.shape[1],pic.shape[2]))
    while i < v :
        picnew[:,i,:]=pic[:,v-i-1,:]
        i=i+1
    return picnew

In [ ]:
def mirrorHor(pic) :
    h=pic.shape[0]
    i=0
    picnew=np.zeros((pic.shape[0],pic.shape[1],pic.shape[2]))
    while i < h :
        picnew[i,:,:]=pic[h-i-1,:,:]
        i=i+1
    return picnew

In [ ]:
def mirrorImages (ppm_filelist):
    for idx, filepath in enumerate(ppm_filelist):
        p=os.path.dirname(filepath)
        vertical=['00013','00012','00034','00033']
        horrizontal=['00012']
        vertAndHorr=['00012']
        if any(v in p for v in vertical):
            pic = plt.imread(filepath)
            newpic=mirrorVer(np.asarray(pic))
            directory =p+'Mv'
            im = Image.fromarray(np.uint8(newpic))
            if not os.path.exists(directory):
                os.makedirs(directory)
            im.save(directory+'\\'+os.path.split(filepath)[1])

        if any(h in p for h in horrizontal):
            pic = plt.imread(filepath)
            newpic=mirrorHor(np.asarray(pic))
            directory =p+'Mh'
            im = Image.fromarray(np.uint8(newpic))
            if not os.path.exists(directory):
                os.makedirs(directory) 
            im.save(directory+'\\'+os.path.split(filepath)[1])
            
        if any(vh in p for vh in vertAndHorr):
            pic = plt.imread(filepath)
            newpic=mirrorVer(np.asarray(pic))
            newpic=mirrorHor(newpic)
            directory =p+'Mhv'
            im = Image.fromarray(np.uint8(newpic))
            if not os.path.exists(directory):
                os.makedirs(directory) 
            im.save(directory+'\\'+os.path.split(filepath)[1])   

In [ ]:
def transform_csv_vert(df: pd.DataFrame) -> pd.DataFrame:
    columns=['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId']
    matrix = df.as_matrix(columns=columns)
    matrix.shape
    matrix2 = np.zeros(matrix.shape)
    for i, row in enumerate(matrix):
        matrix2[i] =[row[0],row[1],row[0]-row[2],row[3],row[0]-row[4],row[5], row[6]]
    matrix2 = matrix2.astype(int)
    df2 = pd.DataFrame(matrix2, columns=columns)
    df2['Filename'] = df['Filename']
    df2 = df2[['Filename'] + columns]
    return df2

In [ ]:
def transform_csv_horr(df: pd.DataFrame) -> pd.DataFrame:
    columns=['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId']
    matrix = df.as_matrix(columns=columns)
    matrix.shape
    matrix2 = np.zeros(matrix.shape)
    for i, row in enumerate(matrix):
        matrix2[i] =[row[0],row[1],row[2],row[1]-row[3],row[4],row[1]-row[5], row[6]]
    matrix2 = matrix2.astype(int)
    df2 = pd.DataFrame(matrix2, columns=columns)
    df2['Filename'] = df['Filename']
    df2 = df2[['Filename'] + columns]
    return df2

In [ ]:
def mirrorBBox (csv_filelist):
    for filepath in csv_filelist:
        df = pd.read_csv(filepath, sep=';')
        
        p=os.path.dirname(filepath)
        vertical=['00013','00012','00034','00033']
        horrizontal=['00012']
        vertAndHorr=['00012']
        if any(v in p for v in vertical):
            df2 = transform_csv_vert(df)
            directory =p+'Mv'
            if not os.path.exists(directory):
                os.makedirs(directory)
            df2.to_csv(directory+'\\'+os.path.split(filepath)[1], sep=';', index=False)

        if any(h in p for h in horrizontal):
            df2 = transform_csv_horr(df)
            directory =p+'Mh'
            if not os.path.exists(directory):
                os.makedirs(directory) 
            df2.to_csv(directory+'\\'+os.path.split(filepath)[1], sep=';', index=False)
            
        if any(vh in p for vh in vertAndHorr):
            df2 = transform_csv_vert(df)
            df2 = transform_csv_horr(df2)
            directory =p+'Mhv'
            if not os.path.exists(directory):
                os.makedirs(directory) 
            df2.to_csv(directory+'\\'+os.path.split(filepath)[1], sep=';', index=False)

In [ ]:
mirrorImages(ppm_filelist)
mirrorBBox(csv_filelist)